# Import Modules & Download required files

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
from datetime import datetime
import time
import os
import glob
import dlib
import threading
import math
!pip install pytesseract
import pytesseract
# below installation needed to run pytesseract
!sudo apt-get install tesseract-ocr

print("Required modules imported")
# create required output directory
!mkdir output
!mkdir output/overspeedVehicles

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=f0ace897eb5f4aaec503fdf4574d88f6581ec910af6db3691bc648d6aeb6fd82
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http:/

In [2]:
!wget https://www.dropbox.com/s/g19fvht674q219r/cars.mp4
!wget https://www.dropbox.com/s/bkwmskdd10v9vh1/myhaar.xml
print("Required files downloaded")

--2021-12-26 14:50:39--  https://www.dropbox.com/s/g19fvht674q219r/cars.mp4
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/g19fvht674q219r/cars.mp4 [following]
--2021-12-26 14:50:40--  https://www.dropbox.com/s/raw/g19fvht674q219r/cars.mp4
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1bd49a6d1c0256d6597f1cd5a0.dl.dropboxusercontent.com/cd/0/inline/BckP9DvzvhLxyx1nKasFgYJ7fC6PrA2wOkHaeHcqySYV2-Lv33fEg6F-o-tX8NDWDXBIdWUtrYRkhbO0JZ5B0M0Co4i4xWKEoBWOUDGnnoIN9W7k8z3xt9z_KIrBKovwmqzkKz_ymzG1SLRy_U6id6oT/file# [following]
--2021-12-26 14:50:40--  https://uc1bd49a6d1c0256d6597f1cd5a0.dl.dropboxusercontent.com/cd/0/inline/BckP9DvzvhLxyx1nKasFgYJ7fC6PrA2wOkHaeHcqySYV2-Lv33fEg6F-o-tX8NDWDXBIdWUtrYRkhbO0JZ5B0M0Co4i4xWKEoBW

# Initialize Vehicle detect and speed estimation model

In [4]:
video = cv2.VideoCapture('cars.mp4')

WIDTH = 1280
HEIGHT = 720
# myhaar.xml is pretrained model on images of cars from the rear
carCascade = cv2.CascadeClassifier('myhaar.xml')
############################################################################

# function to estimate speed
def estimateSpeed(location1, location2):
  d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
  # ppm = location2[2] / carWidht
  ppm = 8.8
  d_meters = d_pixels / ppm
  #print("d_pixels=" + str(d_pixels), "d_meters=" + str(d_meters))
  fps = 18
  speed = d_meters * fps * 3.6
  return speed



# Write output to video file  #########<To create output video file>##########
out = cv2.VideoWriter('/content/output/out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (WIDTH,HEIGHT))

# dataframe to record overspeeding vehicles
overspeed = {'carID': [], "speed": [], 'date_time': [],'vehicle_plate': 0 }
overspeedDIR = "/content/output/overspeedVehicles/"

# Function to convert number_plate from image to text

In [5]:
def get_number_plate(filepath):
  """
  take image file location, scansimage for number plate,
   and return number plate as string
  """
  # importing image
  image = cv2.imread(filepath)
  # converting image to greyscale for faster processing
  grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # plt.imshow(image)
  # plt.figure()
  # plt.imshow(grey)

  # creating noise in image by blurring it
  blur = cv2.bilateralFilter(grey, 20, 100, 100)
  # plt.imshow(blur)

  edges = cv2.Canny(blur, 15, 100)
  # plt.imshow(edges)

  # finding contours in image
  # there are three arguments in cv.findContours() function
  # first one is source image, second is contour retrieval mode, third is contour approximation method.
  contours, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) 
  image_copy = image.copy()
  _ = cv2.drawContours(image_copy, contours, -1, (255,0,255),2)
  # plt.imshow(image_copy)

  # high number of contours found, so sorting to get top 20 max area contours
  contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5] 
  _ = cv2.drawContours(image_copy, contours, -1, (255,0,255),2)
  # plt.imshow(image_copy)

  plate = None

  for contour in contours:
    perimeter = cv2.arcLength(contour, True)
    edges_count = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

    if len(edges_count) == 4:
      x,y,w,h = cv2.boundingRect(contour)
      plate = image[y:y+h, x:x+w]
      break

  if type(plate) == type(None):
    return "unable to detect"

  else:
  # plt.imshow(plate)
  # saving detected number plate
    plate_path = filepath.split('.')
    plate_path[0] = plate_path[0] + "_plateDetected"
    plate_path = '.'.join(plate_path)
    cv2.imwrite(plate_path,plate)
    # decode string from number plate

    # tessdata_dir_config = "/usr/share/tesseract-ocr/4.00/tessdata" #, config=tessdata_dir_config
    car_number = pytesseract.image_to_string(plate, lang = 'eng')
    # splitting to remove unwanted characters
    # print(car_number.split('\n')[0])

    if car_number.split('\n')[0] == '\x0c':
      return "unable to detect"
    else:
      return car_number.split('\n')[0]


In [6]:
# get_number_plate("/content/numberplate.jpg")

# Function to detect vehicles & their respective speeds

In [7]:
def detect_vehicle_speed():


  rectangleColor = (0, 255, 0)
  frameCounter = 0
  currentCarID = 0
  fps = 0

  carTracker = {}
  carNumbers = {}
  carLocation1 = {}
  carLocation2 = {}
  speed = [None] * 3000

  # tracking vehicles from images
  while True:

    start_time = time.time()
    rc, image = video.read()
    if type(image) == type(None):
      break

    image = cv2.resize(image, (WIDTH, HEIGHT))
    resultImage = image.copy()

    frameCounter = frameCounter + 1

    carIDtoDelete = []

    for carID in carTracker.keys():
      trackingQuality = carTracker[carID].update(image)

      if trackingQuality < 15:
        carIDtoDelete.append(carID)

    for carID in carIDtoDelete:
      print ('Removing carID ' + str(carID) + ' from list of trackers.')
      # print ('Removing carID ' + str(carID) + ' previous location.')
      # print ('Removing carID ' + str(carID) + ' current location.')
      carTracker.pop(carID, None)
      carLocation1.pop(carID, None)
      carLocation2.pop(carID, None)

    if not (frameCounter % 10):
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))
      

      for (_x, _y, _w, _h) in cars:
        x = int(_x)
        y = int(_y)
        w = int(_w)
        h = int(_h)

        x_bar = x + 0.5 * w
        y_bar = y + 0.5 * h

        matchCarID = None

        for carID in carTracker.keys():
          trackedPosition = carTracker[carID].get_position()

          t_x = int(trackedPosition.left())
          t_y = int(trackedPosition.top())
          t_w = int(trackedPosition.width())
          t_h = int(trackedPosition.height())

          t_x_bar = t_x + 0.5 * t_w
          t_y_bar = t_y + 0.5 * t_h

          if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
            matchCarID = carID

        if matchCarID is None:
          print ('Creating new tracker ' + str(currentCarID))

          tracker = dlib.correlation_tracker()
          tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

          carTracker[currentCarID] = tracker
          carLocation1[currentCarID] = [x, y, w, h]

          currentCarID = currentCarID + 1

    #cv2.line(resultImage,(0,480),(1280,480),(255,0,0),5)


    for carID in carTracker.keys():
      trackedPosition = carTracker[carID].get_position()

      t_x = int(trackedPosition.left())
      t_y = int(trackedPosition.top())
      t_w = int(trackedPosition.width())
      t_h = int(trackedPosition.height())

      cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)

      # speed estimation
      carLocation2[carID] = [t_x, t_y, t_w, t_h]

    end_time = time.time()

    if not (end_time == start_time):
      fps = 1.0/(end_time - start_time)


    ############################################################################## 
    # GET VEHICLE SPEED
    ##############################################################################
    for i in carLocation1.keys():	
      if frameCounter % 1 == 0:
        [x1, y1, w1, h1] = carLocation1[i]
        [x2, y2, w2, h2] = carLocation2[i]

        # print('previous location: ', str(carLocation1[i]),', current location: ', str(carLocation2[i]))
        carLocation1[i] = [x2, y2, w2, h2]

      # print('new previous location: ', str(carLocation1[i]))
        if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
          if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
            speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])

      #if y1 > 275 and y1 < 285:
          if speed[i] != None and y1 >= 180:
            cv2.putText(resultImage, str(int(speed[i])) + " km/hr", (int(x1 + w1/2), int(y1-5)),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)
            print(f"car ID: {carID}, speed: {speed[i]} KMPH")

            # SETTING SPEED LIMIT OF 60 KMPH FOR OVERSPEEDING
            if speed[i] > 60:

              # saving cropped vehicle image of overspeed vehicle
              cropImg = image[y1:y1+h1, x1:x1+w1]
              loc = overspeedDIR + str(carID) + ".jpg"
              cv2.imwrite(loc, cropImg)

              # calling function to detect number plate
              numberplate = get_number_plate(filepath = loc)


              overspeed['carID'].append(carID)
              overspeed['speed'].append(int(speed[i]))
              overspeed['date_time'].append(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
              overspeed['vehicle_plate'] = numberplate


    # cv2.imshow('result', resultImage)
    # Write the frame into the file 'output.avi'
    out.write(resultImage)


  # if output video is enabled, release by uncommenting below
  out.release()
  video.release()

  cv2.destroyAllWindows()


# Execution

In [8]:
detect_vehicle_speed()

Creating new tracker 0
Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Removing carID 2 from list of trackers.
Creating new tracker 4
Creating new tracker 5
Removing carID 5 from list of trackers.
Removing carID 4 from list of trackers.
Removing carID 3 from list of trackers.
Creating new tracker 6
Creating new tracker 7
Creating new tracker 8
Removing carID 6 from list of trackers.
car ID: 8, speed: 66.68056328992098 KMPH
car ID: 8, speed: 66.68056328992098 KMPH
car ID: 8, speed: 66.68056328992098 KMPH
Creating new tracker 9
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
Removing carID 1 from list of trackers.
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KMPH
car ID: 9, speed: 66.68056328992098 KM

# Storing overspeed vehicles data in csv file

In [9]:
# overspeed_dataframe = pd.DataFrame(overspeed).drop_duplicates(keep = 'last')
overspeed_dataframe = pd.DataFrame(overspeed).groupby('carID').tail(1).reset_index(drop = True)
overspeed_dataframe.to_csv("/content/output/overspeedinglogs.csv", index = False)
overspeed_dataframe

,carID,speed,date_time,vehicle_plate
0,8,66,2021-12-26 14:52:39,unable to detect
1,9,66,2021-12-26 14:52:41,unable to detect
2,10,66,2021-12-26 14:52:43,unable to detect
3,37,98,2021-12-26 14:52:55,unable to detect
4,70,63,2021-12-26 14:53:16,unable to detect
5,69,63,2021-12-26 14:53:16,unable to detect
6,68,63,2021-12-26 14:53:17,unable to detect
7,74,63,2021-12-26 14:53:18,unable to detect
8,153,66,2021-12-26 14:53:55,unable to detect
9,157,66,2021-12-26 14:53:58,unable to detect


In [10]:
# import os
# import zipfile
# zf = zipfile.ZipFile("output.zip", "w")
# for dirname, subdirs, files in os.walk(r"/content/output"):
#     zf.write(dirname)
#     for filename in files:
#         zf.write(os.path.join(dirname, filename))
# zf.close()